In [46]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

# using Pkg
# Pkg.update()
# Pkg.add("Convex")
# Pkg.add("SCS")
# Pkg.add("JuMP")
# Pkg.add("ECOS")
# Pkg.add("Plots")
# Pkg.add("LinearAlgebra")
# Pkg.add("ControlSystems")

  Activating project at `~/Documents/eth_courses/notebooks/optimization`
    Updating registry at `~/.julia/registries/General.toml`
  No Changes to `~/Documents/eth_courses/notebooks/optimization/Project.toml`
  No Changes to `~/Documents/eth_courses/notebooks/optimization/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Documents/eth_courses/notebooks/optimization/Project.toml`
  No Changes to `~/Documents/eth_courses/notebooks/optimization/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Documents/eth_courses/notebooks/optimization/Project.toml`
  No Changes to `~/Documents/eth_courses/notebooks/optimization/Manifest.toml`
   Resolving package versions...
   Installed StrideArraysCore ─ v0.4.13
   Installed Polynomials ────── v3.2.10
    Updating `~/Documents/eth_courses/notebooks/optimization/Project.toml`
  [a6e380b2] + ControlSystems v1.7.2
    Updating `~/Documents/eth_courses/notebooks/optimization/Manifest.toml`
  [621f4979] + AbstractFFTs v

  ✓ DelayDiffEq
  ✓ ControlSystems
  18 dependencies successfully precompiled in 386 seconds. 247 already precompiled.


In [47]:
using LinearAlgebra, Plots

import Convex as cvx 
import ECOS
using Random
using ControlSystems

Random.seed!(1)

TaskLocalRNG()

## Least Squares

In [41]:
# Generate random problem data
m = 4;  n = 5
A = randn(m, n); b = randn(m)

# Create a (column vector) variable of size n x 1.
x = Variable(n)

# The problem is to minimize ||Ax - b||^2 subject to x >= 0

# objective
problem = cvx.minimize(sumsquares(A * x - b))

# constrains
problem.constraints += (x >= 0)

# Solve the problem by calling solve!
solve!(problem, ECOS.Optimizer; silent_solver = true)

# Check the status of the problem
problem.status # :Optimal, :Infeasible, :Unbounded etc.

# Get the optimum value
problem.optval

## Quadratic form

In [43]:
n = 10 
Q = randn(n,n); Q = Q'*Q + I # create PSD matrix 
q = randn(n)

A = randn(3,n)
b = randn(3)

x = cvx.Variable(n)

# NOTE: quadform(x,Q) = x'*Q*x 
cost = 0.5*cvx.quadform(x,Q) + dot(q,x) 

prob = cvx.minimize(cost)

prob.constraints += (A*x == b)

cvx.solve!(prob, ECOS.Optimizer; silent_solver = false)

xcvx = x.value::Matrix # This will always be a matrix
xcvx = vec(x.value) # convert to vector easily 


ECOS 2.0.8 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +5.050e-02  -1.770e+01  +6e+01  4e-01  2e-01  1e+00  2e+01    ---    ---    1  2  - |  -  - 
 1  -1.611e-01  -9.488e-01  +5e+00  2e-02  1e-02  3e-01  2e+00  0.9318  3e-02   2  2  2 |  0  0
 2  -9.053e-02  -5.021e-01  +3e+00  2e-02  9e-03  7e-01  9e-01  0.7106  4e-01   2  2  2 |  0  0
 3  +1.372e-01  +9.362e-02  +2e-01  1e-03  6e-04  2e-02  6e-02  0.9646  3e-02   2  2  2 |  0  0
 4  +1.473e-01  +1.433e-01  +2e-02  1e-04  5e-05  1e-03  5e-03  0.9096  3e-04   2  2  2 |  0  0
 5  +1.471e-01  +1.461e-01  +5e-03  3e-05  1e-05  4e-04  1e-03  0.7656  4e-02   2  2  2 |  0  0
 6  +1.469e-01  +1.467e-01  +2e-03  8e-06  4e-06  2e-04  5e-04  0.7559  1e-01   3  2  1 |  0  0
 7  +1.470e-01  +1.470e-01  +7e-05  4e-07  2e-07  9e-06  2e-05  0.9741  2e-02   2  2  2 |  0  0
 8  +1.470e-01  +1.470e-01  +7e-06  4e-08  2e-

10-element Vector{Float64}:
  0.33743916971690374
  0.3000951281071971
  0.21102528613241286
 -0.19490172865010272
 -0.07648787610139045
 -0.06138807405624016
 -0.3858348483991579
  0.19465518710571525
 -0.0050046404648325
 -0.06940389429198414

## Convex Trajectory Optimization

In [60]:
function is_controllable(A,B)
    n = size(A,1)
    return rank(ctrb(A,B)) == n 
end

let 
    
    # create linear system
    nx = 4 
    nu = 2 
    A = randn(nx,nx);
    B = randn(nx,nu);
    @assert is_controllable(A,B)
    
    # time steps 
    N = 20 
    x_ic = randn(nx)
    x_g = randn(nx)
    
    # terminal cost 
    Qf = randn(nx,nx); Qf = Qf'*Qf + I # make PSD Qf 
    
    # create cvx variables x_k = X[:,k], u_k = U[:,k]
    X = cvx.Variable(nx, N)
    U = cvx.Variable(nu, N - 1)
    
    # create cost 
    cost = 0 
    for k = 1:(N-1)
        xk = X[:,k]
        uk = U[:,k]
        cost += cvx.sumsquares(xk - x_g)
        cost += norm(uk, 1)
    end
    xn = X[:,N]
    cost += 0.5*cvx.quadform(xn, Qf)
    
    # initialize cvx problem 
    prob = cvx.minimize(cost)
    
    # initial condition constraint 
    prob.constraints += X[:,1] == x_ic 
    
    for k = 1:(N-1)
        # dynamics constraints 
        prob.constraints += (X[:,k+1] == A*X[:,k] + B*U[:,k])
    end
    
    # goal constraint 
    prob.constraints += X[:,N] == x_g
    
    # norm(u)<3 
    for k = 1:(N-1)
        uk = U[:,k]
        prob.constraints += norm(uk,2) <= 3 
    end
    
    x_min = -20*ones(nx)
    x_max =  20*ones(nx)
    for k = 1:N
        xk = X[:,k]
        prob.constraints += xk <= x_max 
        prob.constraints += xk >= x_min 
    end
    
    # solve problem (silent solver tells us the output)
    cvx.solve!(prob, ECOS.Optimizer; silent_solver = false)
    
    if prob.status != cvx.MathOptInterface.OPTIMAL
        error("Convex.jl problem failed to solve for some reason")
    end
        
    # convert the solution matrices into vectors of vectors 
    X = X.value::Matrix
    U = U.value::Matrix
end


ECOS 2.0.8 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -3.631e+03  +4e+03  6e-02  5e-01  1e+00  1e+01    ---    ---    1  2  - |  -  - 
 1  +3.176e+01  -1.381e+03  +2e+03  2e-02  2e-01  1e+00  5e+00  0.6413  8e-02   1  1  1 |  0  0
 2  +6.079e+01  -1.264e+03  +2e+03  2e-02  1e-01  2e+00  5e+00  0.2416  6e-01   1  1  2 |  0  0
 3  +8.255e+01  -1.102e+03  +1e+03  2e-02  7e-02  2e+00  4e+00  0.2729  5e-01   1  1  2 |  0  0
 4  +1.467e+02  -6.349e+02  +9e+02  1e-02  3e-02  3e+00  3e+00  0.8781  6e-01   1  1  1 |  0  0
 5  +1.179e+02  -3.648e+02  +6e+02  7e-03  2e-02  2e+00  2e+00  0.4621  2e-01   2  1  1 |  0  0
 6  +1.178e+02  -2.806e+02  +5e+02  6e-03  1e-02  1e+00  1e+00  0.3916  5e-01   2  2  2 |  0  0
 7  +1.138e+02  -2.001e+02  +4e+02  4e-03  8e-03  1e+00  1e+00  0.3927  4e-01   2  2  2 |  0  0
 8  +9.539e+01  +3.851e+00  +1e+02  1e-03  2e-

2×19 Matrix{Float64}:
 -3.01176e-9  -2.87701e-10  -4.38317e-10  …  -3.48157e-13  -4.06267e-10
  0.788816     0.0250292    -0.0684618       -0.160314      0.141862